In [2]:
import numpy as np
import matplotlib.pyplot as plt
import RNG.Distributions as dst
import math
import scipy.stats as sstats

## Exercise 6 Task 1)



In [12]:
def truncatePoiss(A,i):
    return (A**i)/math.factorial(i)

def norm(i,mu,sigma2):
    return np.exp(-1*(i-mu)**2/(2*sigma2))


samples = []
n_warmup = 100
n_samples = 1000
x = 0
m = 11
A = 8.0

hxy_s = lambda x: np.random.uniform(-100,100)
hxy = lambda x,y: 1
g = lambda i: norm(i,0,1)


P_is = np.array([g(i) for i in range(m)])
P_is = P_is


def MetropolisHastings(n_samples,g,hxy,hxy_sampler, x_start = 0):
    samples = []
    x = x_start
    for _ in range(n_samples):
        y  = hxy_sampler(x)
        P_x = g(x)
        P_y = g(y)
        P_xy = hxy(x,y)
        P_yx = hxy(y,x)

        if (np.random.uniform(0,1)<min(1,(P_y*P_yx)/(P_x*P_xy))):
            x = y

        samples += [x]

    return samples




samples = MetropolisHastings(1000000, g, hxy, hxy_s)


In [15]:
def bootStrap(data_vector):
    U_disc = np.random.randint(0,len(data_vector),len(data_vector))
    return [data_vector[u] for u in U_disc]

n = len(samples)
X_i = samples
X_bar = []
MSE_sample = []

for r in range(100):
    X_sample = bootStrap(X_i)
    X_bar += [np.mean(X_sample)]
    MSE_sample += [np.sum((np.array(X_sample) - np.mean(X_sample))**2)/n**2]


mu = np.mean(X_bar)
print(f"This is the expectation of our MSE: {np.mean(MSE_sample)}")


X = (X_i.sum()/n - mu)
p = ((X>-5) & (X<5)).sum()/n
print(f"This is our empirical estimate of the given probability:{p}")

This is the expectation of our MSE: 9.986589415663484e-07


AttributeError: 'list' object has no attribute 'sum'

In [130]:
def MetropolisHastingsMatrix(n_samples,g,hxy,hxy_sampler, sample_shape, x_start = 0):
    samples = np.zeros(sample_shape)
    x = x_start
    for _ in range(n_samples):
        y  = hxy_sampler(x)
        P_x = g(x)
        P_y = g(y)
        P_xy = hxy(x,y)
        P_yx = hxy(y,x)

        if (np.random.uniform(0,1)<min(1,(P_y*P_yx)/(P_x*P_xy))):
            x = y

        samples[x[0],x[1]] += 1

    return samples

In [185]:
samples = []
n_samples = 1000
m = 10
A = 8.0

hxy_s = lambda A: np.random.randint(0,m+1)
hxy = lambda A,B: 1
g = lambda A: truncatePoiss(4,A[0])*truncatePoiss(4,A[1]) if 0<= A[0]+A[1] <= m else 0


P_is = np.zeros((m+1,m+1))

for i in range(m+1):
    for j in range(m+1):
        P_is[i,j] += g([i,j])

P_is = np.array(P_is)


samples = MetropolisHastingsMatrix(100000, g, hxy, hxy_s,(11,11), x_start = (0,0))


sstats.chisquare(samples, f_exp=P_is)

TypeError: 'tuple' object does not support item assignment

In [18]:
def Gibbs(nr_samples, P_cond,j = 0, samples_shape = (11,11)):

    samples = np.zeros(samples_shape)
    for _ in range(nr_samples):

        X = P_cond(j)
        i = np.random.choice([i for i in range(len(X))],p=X)
        X = P_cond(i)
        j = np.random.choice([i for i in range(len(X))],p=X)
        samples[i,j] += 1

    return samples

m = 10
def P_cond(j):
    X = []
    denominator = 0

    for x in range(m - j + 1):
        denominator += truncatePoiss(4,x)

    for i in range(m - j + 1):
        X += [truncatePoiss(4,i)/denominator]

    return X

samples_gibbs = Gibbs(100000,P_cond)
P_empir_gibbs = samples_gibbs